In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
from matplotlib.transforms import Bbox
import seaborn as sns
import scanpy as sc
import scipy
import sklearn

from importlib import reload

import plotting as plt2

In [2]:
fpath = "/scratch/indikar_root/indikar1/cstansbu/HSC/geneformer_adata/processed.anndata.lt.h5ad"

adata = sc.read_h5ad(fpath)
adata

AnnData object with n_obs × n_vars = 54347 × 19070
    obs: 'n_counts', 'cell_type', 'dataset', 'organ_tissue'
    var: 'gene_name', 'ensemble_id', 'n_counts'
    uns: 'go_annotations', 'log1p', 'panglaodb'
    layers: 'log_norm', 'raw_counts'

In [3]:
mask = (adata.obs['cell_type'] == 'HSC') & (adata.obs['dataset'] == 'young2_HSC')
weng_HSC = adata[mask, :]
print(weng_HSC)

# get raw counts
expression_df = weng_HSC.to_df(layer='raw_counts')
signature = expression_df.sum(axis=0)
signature = signature.reset_index()
signature.columns = ['gene_name', 'sum_count']

# mean log TPM
mean_log_tpm = weng_HSC.to_df().mean(axis=0)
signature['mean_log_TPM'] = mean_log_tpm.values

name2id = dict(zip(weng_HSC.var['gene_name'].values, weng_HSC.var['ensemble_id'].values))
signature['ensemble_id'] = signature['gene_name'].map(name2id)

outpath = "/home/cstansbu/temp/weng_2024_HSC_raw_counts.csv"

signature.to_csv(outpath, index=False)

signature.head()

View of AnnData object with n_obs × n_vars = 3243 × 19070
    obs: 'n_counts', 'cell_type', 'dataset', 'organ_tissue'
    var: 'gene_name', 'ensemble_id', 'n_counts'
    uns: 'go_annotations', 'log1p', 'panglaodb'
    layers: 'log_norm', 'raw_counts'


,gene_name,sum_count,mean_log_TPM,ensemble_id
0,A1BG,194.0,0.030028,ENSG00000121410
1,A1CF,1.0,0.000736,ENSG00000148584
2,A2M,131.0,0.162159,ENSG00000175899
3,A2ML1,4.0,0.053027,ENSG00000166535
4,A3GALT2,4.0,0.011742,ENSG00000184389


In [12]:
# save expression
outpath = "/home/cstansbu/temp/all_counts.csv"

weng_HSC.var_names = weng_HSC.var['ensemble_id'].values
all_counts = weng_HSC.to_df(layer='raw_counts')
all_counts.to_csv(outpath)
all_counts.head()

,ENSG00000121410,ENSG00000148584,ENSG00000175899,ENSG00000166535,ENSG00000184389,ENSG00000128274,ENSG00000118017,ENSG00000094914,ENSG00000081760,ENSG00000114771,...,ENSG00000174442,ENSG00000122952,ENSG00000198205,ENSG00000198455,ENSG00000070476,ENSG00000203995,ENSG00000162378,ENSG00000159840,ENSG00000074755,ENSG00000036549
AAACAGCCAACCCTAA-3_young2_HSC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
AAACAGCCATAGGCGA-3_young2_HSC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
AAACATGCAGGCTGTT-3_young2_HSC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCGCGTCCTTAGT-3_young2_HSC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0
AAACCGCGTCCTTTAA-3_young2_HSC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0


In [8]:
adata.var.head()

,gene_name,ensemble_id,n_counts
gene_name,,,
A1BG,A1BG,ENSG00000121410,1653.0
A1CF,A1CF,ENSG00000148584,53.0
A2M,A2M,ENSG00000175899,471452.0
A2ML1,A2ML1,ENSG00000166535,6200.0
A3GALT2,A3GALT2,ENSG00000184389,1324.0


In [4]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
dict(zip(weng_HSC.var['gene_name'].values, weng_HSC.var['ensemble_id'].values))

In [ ]:
signature = weng_HSC.to_df(layer='raw_counts').sum(axis=0)

In [ ]:
adata.obs['cell_type'].value_counts()

In [ ]:
break

In [ ]:
adata.obs['dataset'].value_counts()

In [ ]:
adata.obs['cell_type'].value_counts()

In [ ]:
# break